In [2]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.metrics import classification_report
from tqdm import tqdm

# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load data from CSV
file_path = 'final_manglish_transliterated.csv'  # Replace with your file path
data = pd.read_csv(file_path)

# Ensure 'commentText' column contains strings
data['transliterated_text'] = data['transliterated_text'].astype(str)

# Function to convert sentiment labels to numerical values
sentiment_dict = {
    'Positive': 0,
    'Negative': 1,
    'Not_relevant': 2,
    'Mixed Feelings': 3,
    'Neutral': 4
}

data['Sentiment_Class'] = data['Sentiment_Class'].map(sentiment_dict)

# Upsample minority classes to match the size of the majority class
data_upsampled = pd.concat([
    resample(data[data['Sentiment_Class'] == sentiment_dict['Positive']], replace=True, n_samples=len(data[data['Sentiment_Class'] == sentiment_dict['Not_relevant']])),
    resample(data[data['Sentiment_Class'] == sentiment_dict['Negative']], replace=True, n_samples=len(data[data['Sentiment_Class'] == sentiment_dict['Not_relevant']])),
    resample(data[data['Sentiment_Class'] == sentiment_dict['Not_relevant']], replace=True, n_samples=len(data[data['Sentiment_Class'] == sentiment_dict['Not_relevant']])),
    resample(data[data['Sentiment_Class'] == sentiment_dict['Mixed Feelings']], replace=True, n_samples=len(data[data['Sentiment_Class'] == sentiment_dict['Not_relevant']])),
    resample(data[data['Sentiment_Class'] == sentiment_dict['Neutral']], replace=True, n_samples=len(data[data['Sentiment_Class'] == sentiment_dict['Not_relevant']]))
])

# Tokenization and encoding using BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize and encode the texts
encoded_data = tokenizer(list(data_upsampled['transliterated_text']), padding=True, truncation=True, max_length=128, return_tensors='pt')

# Prepare target variable for upsampled data
labels_upsampled = torch.tensor(data_upsampled['Sentiment_Class'].values)

# Split the upsampled data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(encoded_data.input_ids, labels_upsampled, test_size=0.05, random_state=0)

# Create DataLoader for training and testing data
train_dataset = TensorDataset(X_train, y_train)
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=32)

test_dataset = TensorDataset(X_test, y_test)
test_dataloader = DataLoader(test_dataset, batch_size=32)

# Build and move BERT model to GPU
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(sentiment_dict))
model.to(device)

# Fine-tune the model with progress bar
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss()

epochs = 3
for epoch in range(epochs):
    model.train()
    for batch in tqdm(train_dataloader, desc=f'Epoch {epoch+1}/{epochs}'):
        batch = tuple(t.to(device) for t in batch)
        input_ids, labels = batch
        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

# Evaluate the model on test data
model.eval()
y_pred_labels = []
y_true_labels = []
for batch in tqdm(test_dataloader, desc='Evaluating'):
    batch = tuple(t.to(device) for t in batch)
    input_ids, labels = batch
    with torch.no_grad():
        outputs = model(input_ids=input_ids)
        logits = outputs.logits
    predictions = torch.argmax(logits, dim=1)
    y_pred_labels.extend(predictions.tolist())
    y_true_labels.extend(labels.tolist())
    
# Create reverse mapping of sentiment_dict
reverse_sentiment_dict = {v: k for k, v in sentiment_dict.items()}

# Move labels back to CPU for classification report
y_pred_labels = [reverse_sentiment_dict[label] for label in y_pred_labels]
y_true_labels = [reverse_sentiment_dict[label] for label in y_true_labels]

# Print complete classification report
print(classification_report(y_true_labels, y_pred_labels, target_names=sentiment_dict.keys()))


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Evaluating: 100%|██████████| 50/50 [00:46<00:00,  1.07it/s]


                precision    recall  f1-score   support

      Positive       0.97      1.00      0.98       337
      Negative       0.89      0.94      0.91       272
  Not_relevant       0.91      0.89      0.90       320
Mixed Feelings       0.89      0.84      0.86       319
       Neutral       0.90      0.90      0.90       324

      accuracy                           0.91      1572
     macro avg       0.91      0.91      0.91      1572
  weighted avg       0.91      0.91      0.91      1572

